In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2

# %load init.ipy
import os, sys, logging, datetime, warnings, shutil
from importlib import reload

import numpy as np
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import kalepy as kale
import kalepy.utils
import kalepy.plot
# The `nbshow` command runs `plt.show()` in interactive jupyter notebooks, but closes
#   figures when run from the command-line (notebooks are converted to scripts and run as tests)
from kalepy.plot import nbshow

import warnings
# warnings.simplefilter("error")

In [ ]:
fname = "/Users/lzkelley/Programs/kalepy/kalepy/tests/performance/pt-0.1/pt-0.1.npz"
data = np.load(fname)['data'][...]
print(f"Loaded {data.shape=}. {data.size=}")

In [ ]:
def test_performance(data, smaller=10, param=0):
    if smaller is not None:
        data = [dd[slice(None, None, smaller)] for dd in data]

    if param is not None:
        data = data[param]
        
    data = np.asarray(data)
    kde = kale.KDE(data)
    kde.density()
    return

# %timeit test_performance(data, smaller=10, param=0)

In [ ]:
%timeit test_performance(data, smaller=10, param=0)

In [ ]:
%timeit test_performance(data, smaller=10, param=0)

In [ ]:
%timeit test_performance(data, smaller=10, param=0)

In [ ]:
def test_performance(data, smaller=50):
    if smaller is not None:
        data = [dd[slice(None, None, smaller)] for dd in data]

    data = np.asarray(data)
    kde = kale.KDE(data)
    corner = kale.Corner(data.shape[0])
    corner.clean(kde)
    
    samp = kde.resample()
    corner.plot_data(samp)
    
    nbshow()
    return

%time test_performance(data)

In [ ]:
import kalepy.tests
import kalepy.tests.test_kernels

In [ ]:
%time kale.tests.test_kernels.test_kernels_evaluate_nd()

In [ ]:
import time


def setup(num):
    data = kale.utils._random_data_3d_03(num=num)
    reflect = [[0, None], None, [None, 1]]
    kde_clear = kale.KDE(data)
    kde_reflect = kale.KDE(data, reflect=reflect)
    xx = [kale.utils.minmax(xx, stretch=0.2) for xx in data]
    xx = [np.linspace(*xx, 10) for xx in xx]
    xx = kale.utils.meshgrid(*xx)
    xx = [xx.flatten() for xx in xx]
    xx = np.array(xx)
    return xx, kde_clear, kde_reflect


def _test_each(xx, kde):
    times = []
    t0 = time.time()

    _, pdf = kde.density(xx)
    t1 = time.time()

    samp = kde.resample()
    t2 = time.time()
    
    return [t1-t0, t2-t1]
    

def test(xx, clear, reflect, trials):
    retval = []
    for ii in range(trials):
        rv1 = _test_each(xx, clear)
        rv2 = _test_each(xx, reflect)
        retval.append([rv1, rv2])

    return np.array(retval)


def run_test(xx, kdes, num, trials):
    beg = time.time()
    retval = test(xx, *kdes, trials)
    print(f"{num=:.1e}, {trials=} :: done after {time.time()-beg:.2e}s")
    retval = np.percentile(retval, [0, 50, 100], axis=0)
    names = ['clear', 'reflect']
    for ii, name in enumerate(names):
        print(f"{ii}: {name}")
        for jj, label in enumerate(['density', 'resample']):
            msg = f"\t{jj}: {label:10s} "
            for kk, perc in enumerate(['best', 'med']):
                val = retval[kk, ii, jj]
                msg += f"{perc:4s} = {val:.2e}   "
            print(msg)
            
    return

In [ ]:
NUM = 1e4
TRIALS = 10
xx, *kdes = setup(NUM)

In [ ]:
def pdf(xx):
    return (70.0/32.0) * np.power(1 - xx*xx, 3)

def cdf(xx):
    yy = (70/32) * (((-1.0/7.0) * (xx**7)) + ((3.0/5.0) * (xx**5)) - (xx**3) + xx)
    return yy

xx = np.linspace(0.0, 1.0, 1000)
yy = cdf(xx)
yy[-1] = 1.0
pp = pdf(xx)

inv = sp.interpolate.interp1d(yy, xx)

plt.plot(xx, yy)
plt.plot(xx, inv(yy))
plt.plot(xx, pp)

zz = np.random.uniform(0, 1, 10000)
zz = inv(zz)


plt.hist(zz, bins=30, rwidth=0.8, alpha=0.5, histtype='step', density=True)
plt.hist(zz, bins=30, rwidth=0.8, alpha=0.5, histtype='step', density=True, cumulative=True)
plt.show()

In [ ]:
reload(kalepy.kernels)
reload(kalepy)
# %load_ext line_profiler
run_test(xx, kdes, NUM, 1)
# from kalepy import KDE

In [ ]:
reload(kale.kernels)
kale.kernels.Gaussian._sample(10, 3).shape

In [ ]:
run_test(xx, kdes, NUM, TRIALS)

In [ ]:
# %load_ext line_profiler
run_test(xx, kdes, NUM, 5)
# from kalepy import KDE

In [ ]:
# %load_ext line_profiler
run_test(xx, kdes, NUM, 5)
# from kalepy import KDE

In [ ]:
# %load_ext line_profiler
run_test(xx, kdes, NUM, 5)
# from kalepy import KDE

In [ ]:
run_test(xx, kdes, NUM, 5)

In [ ]:
# kde = kale.KDE(small)
# kale.kernels.Kernel.density

In [ ]:
%prun run_test(xx, kdes, NUM, TRIALS)